<a href="https://colab.research.google.com/github/Marei33/xAutoML/blob/main/xAutoML_2022_HPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Please make a duplicate(**File-> Save a copy in Drive**) of this collaborative and rename it (xAutoML_2022_HPO_firstname_lastname.ipynb).

# Session objectives





*   The goal of this lab is to get familir with hyperparameter optimization. 
*   Practice random search and grid search.


# Setup

In [1]:
# your wonderful Python  packages :)
import pandas as pd 
import numpy as np
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_rand
import requests





from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, make_scorer


# for ignoring the warnings massages.
import warnings
warnings.filterwarnings("ignore")


# List of homeworks


1. There is no homework for this lab. :)

# What is HPO?



Hyperparameter Optimization or HPO is a method that helps solve the challenge of tuning hyperparameters of machine learning algorithms. 

Outstanding ML algorithms have multiple, distinct and complex hyperparameters that generate an enormous search space. 

Plenty of start-ups choose to use deep learning in the core of their pipelines, and search space in deep learning methods is even larger than traditional ML algorithms. Tuning on a massive search space is a tough challenge.

# Data collection 
The initial stage in any machine learning process might be data gathering, therefore we must collect data. As an example, we will utilize a fantastic and genuine dataset. As a result, please run the following cell to get the dataset.

In [2]:
# downloading data source from  :https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv
# Also If you want to know more about the data, please check this link (https://datahub.io/machine-learning/bank-marketing)
req = requests.get("https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv")

url_content = req.content
csv_file = open('bank_marketing.csv', 'wb')
csv_file.write(url_content)
csv_file.close()
bank_marketing_df=pd.read_csv("./bank_marketing.csv")

# Preprocessing 

In [3]:
# in order to better understand the data , we chaned the column name based on the source (https://datahub.io/machine-learning/bank-marketing)
bank_marketing_df.columns=['age','job', 'marital', 'education', 'default','balance', 'housing', 'loan', 'contact','day', 'month','duration','campaign','pdays','previous', 'poutcome', 'class']

In [4]:
categorical_cloumns = list(bank_marketing_df.select_dtypes(include=['object']).columns)
categorical_cloumns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [5]:
# assuming there is a Data Engineer who takes care of the data and provides us a well-prepared data. ( Think of that the data already encoded)
labelencoder = LabelEncoder()
for col in categorical_cloumns:
  bank_marketing_df[col]=labelencoder.fit_transform(bank_marketing_df[col])
bank_marketing_df.tail(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,class
45206,51,9,1,2,0,825,0,0,0,17,9,977,3,-1,0,3,2
45207,71,5,0,0,0,1729,0,0,0,17,9,456,2,-1,0,3,2
45208,72,5,1,1,0,5715,0,0,0,17,9,1127,5,184,3,2,2
45209,57,1,1,1,0,668,0,0,1,17,9,508,4,-1,0,3,1
45210,37,2,1,1,0,2971,0,0,0,17,9,361,2,188,11,1,1


In [6]:
bank_marketing_df.shape

(45211, 17)

In [7]:
# Get a sample of the data
bank_marketing_df_sample=bank_marketing_df.sample(20000)

# Fit the Gradient Boosting Classifier
in this section we would like to apply Gradient Boosting Classifier algorithm on the dataset. As we do not apply hyperparameter tuping

In [8]:
# let us create our train and test set (it is not good practice but let us keep it simple, feel free to add more set (train,val and test) or apply CV)
y=bank_marketing_df_sample['class']
X=bank_marketing_df_sample.drop('class',1)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.33, random_state = 2021)

In [9]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [10]:
gb_model = GradientBoostingClassifier(random_state=2021).fit(X_train, y_train)

In [11]:
y_pred_gb = gb_model.predict(X_test)

##  Balanced accuracy score 

In [12]:
# what do you thnik about the accuracy
balanced_accuracy_score_gb=balanced_accuracy_score(y_test,y_pred_gb)
print("Balanced accuracy score for Gradient Boosting Classifier model ",balanced_accuracy_score_gb )

Balanced accuracy score for Gradient Boosting Classifier model  0.6878975754826352


# Now, Time to apply HPO in practice

In [59]:
# try to define your search space by changing the following dict.
SEARCH_PARAMS = {'learning_rate': np.logspace(0.01, 1, 3),
                    'criterion': ["mse"],
                    'n_estimators': np.linspace(10, 200, 3).astype("int"),
                    'max_depth': np.linspace(1,5,3).astype("int"),
                    'min_samples_split': np.array([2,5]),
                    'min_samples_leaf': np.array([1,5]), 
                    #'max_features': ["auto"]
                    }

# Grid Search 

In [60]:
from sklearn.model_selection import  GridSearchCV

#Your code start here
# gb_grid=
gb_grid= GridSearchCV(estimator= gb_model, param_grid= SEARCH_PARAMS, scoring='balanced_accuracy', cv=2 , verbose=2)
#Your code end here

In [61]:
gb_grid.fit(X_train, y_train)

Fitting 2 folds for each of 108 candidates, totalling 216 fits
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=105; total time=   0.5s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=105; total time=   0.5s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END criterion=mse, learning_rate=1.02329299228

GridSearchCV(cv=2, estimator=GradientBoostingClassifier(random_state=2021),
             param_grid={'criterion': ['mse'],
                         'learning_rate': array([ 1.02329299,  3.19889511, 10.        ]),
                         'max_depth': array([1, 3, 5]),
                         'min_samples_leaf': array([1, 5]),
                         'min_samples_split': array([2, 5]),
                         'n_estimators': array([ 10, 105, 200])},
             scoring='balanced_accuracy', verbose=2)

In [62]:
gb_grid.best_params_

{'criterion': 'mse',
 'learning_rate': 1.023292992280754,
 'max_depth': 3,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 105}

In [63]:
print(f'Test Accuracy : {gb_grid.score(X_test,y_test)}')

Test Accuracy : 0.7119545676025337


#  Random Search

In [65]:
from sklearn.model_selection import RandomizedSearchCV

#Your code start here
gb_random = RandomizedSearchCV(gb_model, SEARCH_PARAMS, scoring='balanced_accuracy', cv=2 , verbose=2, random_state=2021)
#Your code end here


In [66]:
gb_random.fit(X_train, y_train)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=3, min_samples_leaf=5, min_samples_split=5, n_estimators=105; total time=   1.1s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=3, min_samples_leaf=5, min_samples_split=5, n_estimators=105; total time=   1.1s
[CV] END criterion=mse, learning_rate=3.198895109691398, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=105; total time=   0.5s
[CV] END criterion=mse, learning_rate=3.198895109691398, max_depth=1, min_samples_leaf=1, min_samples_split=2, n_estimators=105; total time=   0.5s
[CV] END criterion=mse, learning_rate=10.0, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=mse, learning_rate=10.0, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=10; total time=   0.1s
[CV] END criterion=mse, learning_rate=1.023292992280754, max_depth=1, min_sampl

RandomizedSearchCV(cv=2,
                   estimator=GradientBoostingClassifier(random_state=2021),
                   param_distributions={'criterion': ['mse'],
                                        'learning_rate': array([ 1.02329299,  3.19889511, 10.        ]),
                                        'max_depth': array([1, 3, 5]),
                                        'min_samples_leaf': array([1, 5]),
                                        'min_samples_split': array([2, 5]),
                                        'n_estimators': array([ 10, 105, 200])},
                   random_state=2021, scoring='balanced_accuracy', verbose=2)

In [67]:
y_pred_gb_random = gb_random.predict(X_test)

In [68]:
gb_random.best_params_

{'n_estimators': 105,
 'min_samples_split': 5,
 'min_samples_leaf': 5,
 'max_depth': 3,
 'learning_rate': 1.023292992280754,
 'criterion': 'mse'}

In [69]:
auc_score_gb_random=balanced_accuracy_score(y_test,y_pred_gb_random)
print("roc_auc score for random forest model tuned by RandomizedSearchCV  ",auc_score_gb_random )

roc_auc score for random forest model tuned by RandomizedSearchCV   0.716155048137507


In [70]:
print(f'Test Accuracy : {gb_random.score(X_test,y_test)}')

Test Accuracy : 0.716155048137507


# Share your thoughts and provide us feedback regarding to this notebook.(Optional)


Write your feedback here: 

# <font color='green'> Finished! </font> 



